In [0]:
%sql
--To check if there are duplicates in table contactos before the join , just for security

SELECT
  id,
  data_criacao_da_lead,
  COUNT(*) AS cnt
FROM sc_gold.contactos_pbs
GROUP BY id, data_criacao_da_lead
HAVING COUNT(*) > 1
ORDER BY cnt DESC, id, data_criacao_da_lead;


In [0]:
%sql
DESCRIBE sc_gold.`leads_pbs`;
-- or
SHOW COLUMNS IN sc_gold.`leads_pbs`;


In [0]:
%sql
SELECT
  created_time AS lead_created_time
FROM sc_silver.leads_pbs
LIMIT 50;


In [0]:
%sql
SELECT
  data_criacao_da_lead
FROM sc_silver.contactos_pbs
LIMIT 50;

In [0]:

%sql

SELECT * FROM sc_gold.campaigns             LIMIT 50;


In [0]:
%sql
SELECT * FROM sc_gold.leads_pbs             LIMIT 50;


In [0]:
%sql

SELECT * FROM sc_gold.contactos_pbs         LIMIT 50;


In [0]:
%sql
SELECT * FROM sc_gold.propostas_realizadas  LIMIT 50;


In [0]:
%sql
SELECT * FROM sc_gold.deals                 LIMIT 50;

In [0]:
%sql
SELECT * FROM sc_gold.contactos_pbs         LIMIT 50;

In [0]:
# Build and run: CREATE OR REPLACE TABLE sc_gold.contactos_pbs_2 AS SELECT ... FROM sc_gold.contactos_pbs

# 1) Read source columns in order
cols = spark.sql("""
  SELECT column_name
  FROM information_schema.columns
  WHERE table_schema = 'sc_gold'
    AND table_name   = 'contactos_pbs'
  ORDER BY ordinal_position
""").toPandas()['column_name'].tolist()

# Optional: columns you do NOT want renamed (e.g., keep id as-is)
skip = set()   # e.g., {'id'}

# 2) Build the select list with aliases
select_exprs = []
for c in cols:
    new = c if (c.endswith('_contactos') or c in skip) else f"{c}_contactos"
    select_exprs.append(f"`{c}` AS `{new}`")

select_sql = ", ".join(select_exprs)

# 3) Build the full CREATE TABLE AS SELECT statement
create_sql = f"""
CREATE OR REPLACE TABLE sc_gold.contactos_pbs_2 AS
SELECT {select_sql}
FROM sc_gold.contactos_pbs
"""

print(create_sql)  # shows exactly what will run
spark.sql(create_sql)

# 4) Quick verification
spark.sql("DESCRIBE TABLE sc_gold.contactos_pbs_2").show(truncate=False)
spark.sql("SELECT * FROM sc_gold.contactos_pbs_2 LIMIT 5").show()


In [0]:
%sql
SELECT * FROM sc_gold.contactos_pbs_2

In [0]:
# Source and target
source_table = "sc_gold.propostas_realizadas"
target_table = "sc_gold.propostas_realizadas2"

# 1) Get columns
cols = spark.sql(f"""
  SELECT column_name
  FROM information_schema.columns
  WHERE table_schema = 'sc_gold'
    AND table_name   = 'propostas_realizadas'
  ORDER BY ordinal_position
""").toPandas()['column_name'].tolist()

# Optional: columns to keep unchanged
skip = set()   # e.g., {'id'}

# 2) Build select list with renamed aliases
select_exprs = []
for c in cols:
    new = c if (c.endswith('_propostas') or c in skip) else f"{c}_propostas"
    select_exprs.append(f"`{c}` AS `{new}`")

select_sql = ", ".join(select_exprs)

# 3) Build CREATE TABLE AS SELECT
create_sql = f"""
CREATE OR REPLACE TABLE {target_table} AS
SELECT {select_sql}
FROM {source_table}
"""

print(create_sql)   # Show what will run
spark.sql(create_sql)

# 4) Verify
spark.sql(f"DESCRIBE TABLE {target_table}").show(truncate=False)
spark.sql(f"SELECT * FROM {target_table} LIMIT 5").show()


In [0]:
%sql
SELECT * FROM sc_gold.propostas_realizadas2

In [0]:
# Source and target
source_table = "sc_gold.deals"
target_table = "sc_gold.deals2"

# 1) Get columns
cols = spark.sql(f"""
  SELECT column_name
  FROM information_schema.columns
  WHERE table_schema = 'sc_gold'
    AND table_name   = 'deals'
  ORDER BY ordinal_position
""").toPandas()['column_name'].tolist()

# Optional: columns to keep unchanged (e.g., primary keys)
skip = set()   # Example: {'id'}

# 2) Build select list with renamed aliases
select_exprs = []
for c in cols:
    new = c if (c.endswith('_deals') or c in skip) else f"{c}_deals"
    select_exprs.append(f"`{c}` AS `{new}`")

select_sql = ", ".join(select_exprs)

# 3) Build CREATE TABLE AS SELECT
create_sql = f"""
CREATE OR REPLACE TABLE {target_table} AS
SELECT {select_sql}
FROM {source_table}
"""

print(create_sql)   # Show generated SQL
spark.sql(create_sql)

# 4) Verify structure and sample data
spark.sql(f"DESCRIBE TABLE {target_table}").show(truncate=False)
spark.sql(f"SELECT * FROM {target_table} LIMIT 5").show()


In [0]:
%sql
SELECT * FROM sc_gold.deals2

In [0]:
# Source and target
source_table = "sc_gold.campaigns"
target_table = "sc_gold.campaigns2"

# 1) Get columns
cols = spark.sql(f"""
  SELECT column_name
  FROM information_schema.columns
  WHERE table_schema = 'sc_gold'
    AND table_name   = 'campaigns'
  ORDER BY ordinal_position
""").toPandas()['column_name'].tolist()

# Optional: columns to keep unchanged (like primary keys)
skip = set()   # Example: {'id'}

# 2) Build select list with renamed aliases
select_exprs = []
for c in cols:
    new = c if (c.endswith('_campaigns') or c in skip) else f"{c}_campaigns"
    select_exprs.append(f"`{c}` AS `{new}`")

select_sql = ", ".join(select_exprs)

# 3) Build CREATE TABLE AS SELECT
create_sql = f"""
CREATE OR REPLACE TABLE {target_table} AS
SELECT {select_sql}
FROM {source_table}
"""

print(create_sql)   # Preview generated SQL
spark.sql(create_sql)

# 4) Verify new table
spark.sql(f"DESCRIBE TABLE {target_table}").show(truncate=False)
spark.sql(f"SELECT * FROM {target_table} LIMIT 5").show()


In [0]:
%sql
SELECT * FROM sc_gold.campaigns2

In [0]:
%sql
SELECT COUNT(*) AS strict_matches
FROM sc_gold.leads_pbs l
JOIN sc_gold.contactos_pbs_2 c
  ON  l.converted_contact = c.id_contactos
  AND l.created_time      = c.data_criacao_da_lead_contactos;


In [0]:
%sql
WITH joined AS (
  SELECT
    l.*,
    c.*
  FROM sc_gold.leads_pbs       AS l
  JOIN sc_gold.contactos_pbs_2 AS c
    ON  l.converted_contact = c.id_contactos
   AND l.created_time      = c.data_criacao_da_lead_contactos
)
SELECT COUNT(*) AS check_count FROM joined;   -- should be ~118329


In [0]:
%sql
-- (optional) ensure you're in the right place
-- USE CATALOG workspace;
-- USE SCHEMA sc_gold;

CREATE OR REPLACE TABLE sc_gold.Features_Table
USING DELTA
AS
SELECT
  l.*,
  c.*
FROM sc_gold.leads_pbs       AS l
JOIN sc_gold.contactos_pbs_2 AS c
  ON  l.converted_contact = c.id_contactos
  AND l.created_time      = c.data_criacao_da_lead_contactos;


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  f.*,
  p.*   -- assumes Propostas columns are already suffixed (e.g., *_propostas)
FROM sc_gold.Features_Table AS f
LEFT JOIN sc_gold.propostas_realizadas2 AS p
  ON f.id_contactos = p.id_contacto_propostas;   -- keep all Feature rows


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  f.*,
  d.*   -- assumes Deals columns are already suffixed (e.g., *_deals)
FROM sc_gold.Features_Table AS f
LEFT JOIN sc_gold.deals2 AS d
  ON f.id_contacto_propostas = d.id_contacto_deals;   -- keep all Feature rows


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  f.*,
  cp.*   -- assumes campaigns columns are already suffixed (e.g., *_campaigns)
FROM sc_gold.Features_Table AS f
LEFT JOIN sc_gold.campaigns2 AS cp
  ON f.campanha_deals = cp.id_campaigns;   -- join via Features_Table.campanha_deals -> campaigns.id_campaigns


In [0]:
%%sql
CREATE OR REPLACE TABLE sc_gold.fact_table AS
SELECT
    l.id,
  l.converted_contact,
  l.created_time,
  c.id_contactos,
  c.data_criacao_da_lead_contactos
FROM sc_gold.leads_pbs AS l
JOIN sc_gold.contactos_pbs_2 AS c
  ON l.converted_contact = c.id_contactos
 AND l.created_time = c.data_criacao_da_lead_contactos;


In [0]:
%%sql
CREATE OR REPLACE TABLE sc_gold.fact_table AS
SELECT
  f.*,
 
    p.id_contacto_propostas,
  p.id_proposta_realizada_propostas

FROM sc_gold.fact_table AS f
LEFT JOIN sc_gold.propostas_realizadas2 AS p
  ON p.id_contacto_propostas = f.id_contactos;


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Fact_table AS
SELECT
  f.*,
  d.id_deals,
  d.id_contacto_deals,
  d.campanha_deals
FROM sc_gold.Fact_table AS f
LEFT JOIN sc_gold.deals2 AS d
  ON f.id_contacto_propostas = d.id_contacto_deals;


In [0]:
# snapshot the current table to a temp view
spark.sql("CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM sc_gold.fact_table")

to_drop = {'converted_contact','data_criacao_da_lead_contactos','id_contacto_propostas','id_contacto_deals'}
cols = [f"`{c}`" for c in spark.table("sc_gold.fact_table").columns if c not in to_drop]

spark.sql(f"""
CREATE OR REPLACE TABLE sc_gold.fact_table
USING DELTA AS
SELECT {', '.join(cols)} FROM _src
""")

spark.sql("DROP VIEW _src")


In [0]:
%sql
-- 1) Summary counts of duplicates on (id, created_time)
WITH d AS (
  SELECT id, created_time, COUNT(*) AS cnt
  FROM sc_gold.leads_pbs
  GROUP BY id, created_time
  HAVING COUNT(*) > 1
)
SELECT
  COUNT(*)                    AS duplicated_keys,          -- number of (id,created_time) combos with >1 row
  SUM(cnt)                    AS rows_in_duplicated_keys,  -- total rows participating in duplicates
  SUM(cnt) - COUNT(*)         AS extra_rows_to_dedup       -- rows to drop if you keep 1 per key
FROM d;

-- 2) See the duplicated keys and their sizes (top 100)
SELECT id, created_time, COUNT(*) AS cnt
FROM sc_gold.leads_pbs
GROUP BY id, created_time
HAVING COUNT(*) > 1
ORDER BY cnt DESC, id
LIMIT 100;


In [0]:
%sql
-- SUMMARY: how many duplicate (id, created_time) keys exist in sc_gold.features ( this is normal , can have multiple proposals)
WITH d AS (
  SELECT id, created_time, COUNT(*) AS cnt
  FROM sc_gold.features_table
  GROUP BY id, created_time
  HAVING COUNT(*) > 1
)
SELECT
  COUNT(*)            AS duplicated_keys,
  SUM(cnt)            AS rows_in_duplicated_keys,
  SUM(cnt) - COUNT(*) AS extra_rows_to_dedup
FROM d;


In [0]:
%sql
-- SUMMARY: how many duplicate (id_propostas, created_time) keys exist
WITH d AS (
  SELECT id, created_time, COUNT(*) AS cnt
  FROM sc_gold.propostas_realizadas
  GROUP BY id, created_time
  HAVING COUNT(*) > 1
)
SELECT
  COUNT(*)            AS duplicated_keys,          -- number of key pairs with >1 row
  SUM(cnt)            AS rows_in_duplicated_keys,  -- total rows across those keys
  SUM(cnt) - COUNT(*) AS extra_rows_to_dedup       -- rows you’d remove if keeping 1 per key
FROM d;

-- DETAILS: which keys are duplicated (top 100 by size)
SELECT id, created_time, COUNT(*) AS cnt
FROM sc_gold.propostas_realizadas
GROUP BY id, created_time
HAVING COUNT(*) > 1
ORDER BY cnt DESC, id
LIMIT 100;

-- OPTIONAL: list the actual duplicate rows
SELECT t.*
FROM sc_gold.propostas_realizadas t
JOIN (
  SELECT id, created_time
  FROM sc_gold.propostas_realizadas
  GROUP BY id, created_time
  HAVING COUNT(*) > 1
) d
ON t.id = d.id AND t.created_time = d.created_time
ORDER BY t.id, t.created_time;


In [0]:
%sql
WITH d AS (
  SELECT
    `id`,
    `id_deals`,
    `id_proposta_realizada_propostas`,
    COUNT(*) AS cnt
  FROM sc_gold.features_table
  GROUP BY `id`, `id_deals`, `id_proposta_realizada_propostas`
  HAVING COUNT(*) > 1
)
SELECT
  COUNT(*)              AS duplicated_keys,
  SUM(d.cnt)            AS rows_in_duplicated_keys,
  SUM(d.cnt) - COUNT(*) AS extra_rows_to_dedup
FROM d AS d;


In [0]:
%sql
WITH d AS (
  SELECT
    `id`,
    `id_deals`,
    `id_proposta_realizada_propostas`,
    COUNT(*) AS cnt
  FROM sc_gold.fact_table
  GROUP BY `id`, `id_deals`, `id_proposta_realizada_propostas`
  HAVING COUNT(*) > 1
)
SELECT
  COUNT(*)              AS duplicated_keys,
  SUM(d.cnt)            AS rows_in_duplicated_keys,
  SUM(d.cnt) - COUNT(*) AS extra_rows_to_dedup
FROM d AS d;


In [0]:
%sql
SELECT * FROM sc_gold.Fact_table

In [0]:
%sql
SELECT * FROM sc_gold.Features_Table

In [0]:
# 1) Build keep list to Ml Models
cols_to_delete = [  # the  names we listed
    "_fivetran_id","id","lead_owner","lead_owner_name","consentimento",
    "id_lead","chave_instalacao_bd_rede","chave_concessao_bd_rede","chave_contrato_concessionario_bd_rede",
    "chave_agrupamento_performance_vendas","chave_agrupamento_cliente",
    "data_de_validade_de_consentimento_hyundai","lead_response_time","converted_date_time",
    "_fivetran_deleted","_fivetran_synced","_fivetran_id_contactos","id_contactos",
    "contacto_owner_contactos","contacto_owner_name_contactos","created_time_contactos",
    "id_contacto_contactos","consentimento_contactos","id_lead_conversao_contactos",
    "data_de_recolha_de_consentimento_hyundai_contactos","chave_concessao_bd_rede_contactos",
    "chave_instalacao_bd_rede_contactos","chave_contrato_concessionario_bd_rede_contactos",
    "chave_agrupamento_performance_vendas_contactos","chave_agrupamento_cliente_contactos",
    "link_centro_consentimento_contactos","_fivetran_deleted_contactos","_fivetran_synced_contactos",
    "_fivetran_id_propostas","id_propostas","created_time_propostas",
    "proposta_realizada_owner_propostas","proposta_realizada_owner_name_propostas",
    "modified_time_propostas","conta_name_propostas","id_contacto_propostas","contacto_name_propostas",
    "deal_name_propostas","created_by_propostas","modified_by_propostas","descricao_apoio_propostas",
    "codigo_da_proposta_propostas","descricao_pintura_propostas","data_de_criacao_da_proposta_propostas",
    "data_de_entrega_da_proposta_propostas","id_conta_propostas","id_negocio_propostas",
    "id_proposta_realizada_propostas","data_de_validade_da_proposta_propostas","codigo_modelo_propostas",
    "sufixo_modelo_propostas","codigo_cor_exterior_propostas","codigo_cor_interior_propostas",
    "concessionario_owner_propostas","data_prevista_matricula_propostas","data_prevista_de_entrega_propostas",
    "data_da_conclusao_propostas","instalacao_propostas","chave_instalacao_bd_rede_propostas",
    "chave_concessao_bd_rede_propostas","cod_proposta_propostas","codigo_fabricante_cor_interior_propostas",
    "codigo_fabricante_cor_exterior_propostas","_fivetran_deleted_propostas","_fivetran_synced_propostas",
    "_fivetran_id_deals","deal_owner_name_deals","id_contacto_deals",
    "closing_date_deals","deal_owner_deals","conta_name_deals","contacto_name_deals","id_negocio_deals",
    "data_prevista_de_reentrada_em_negocio_deals","id_conta_deals","codigo_negocio_deals","data_venda_deals",
    "nome_contacto_deals","id_deals","data_decisao_negocio_deals","_fivetran_deleted_deals",
    "_fivetran_synced_deals","_fivetran_id_campaigns","id_campaigns",
    "campaign_owner_name_campaigns","created_time_campaigns","codigo_campanha_campaigns",
    "_fivetran_deleted_campaigns","_fivetran_synced_campaigns"
]

src_tbl = "sc_gold.Features_Table"

# compute keep list from live schema
all_cols = spark.table(src_tbl).columns
keep_cols = [c for c in all_cols if c not in cols_to_delete]
keep_sql = ", ".join([f"`{c}`" for c in keep_cols])  # quote names safely

# 2) Snapshot and recreate from projection
spark.sql(f"CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM {src_tbl}")
spark.sql(f"""
CREATE OR REPLACE TABLE {src_tbl}
USING DELTA AS
SELECT {keep_sql} FROM _src
""")
spark.sql("DROP VIEW _src")

#You should generally remove columns that:
#Are unique identifiers (e.g., _fivetran_id, id, id_lead, id_contactos, id_propostas, id_deals, id_campaigns).
#Are timestamps or sync markers from ETL (_fivetran_synced, _fivetran_deleted, etc.).
#Contain names of people or owners (lead_owner_name, campaign_owner_name_campaigns, etc.).
#Are free-text codes or descriptive identifiers (codigo_campanha_campaigns, codigo_da_proposta_propostas, etc.).
#Are internal foreign keys / lookup fields (chave_instalacao_bd_rede, chave_agrupamento_cliente, etc.).
#Are emails / links (link_centro_consentimento_contactos).




In [0]:
# 1) Built the second delete list after a better analysis 
cols_to_delete = [
    # --- timestamps / dates ---
  
    "data_criacao_da_lead_contactos",
    "data_prevista_de_entrega_deals",
    "data_prevista_matricula_deals",
    "data_criacao_da_lead_deals",

    # --- IDs / keys ---
    "created_by", "modified_by",
    "id_classe_propostas", "id_model_group_propostas",
    "id_combustivel_propostas", "id_modelo_propostas",
    "chave_agrupamento_cliente_propostas",
    "tylacode_propostas",
    "chave_concessao_bd_rede_deals", "chave_instalacao_bd_rede_deals",
    "chave_contrato_concessionario_bd_rede_deals",
    "chave_agrupamento_performance_vendas_deals", "chave_agrupamento_cliente_deals","campanha_deals",

    # --- descriptive text ---
    "descricao_model_group_propostas",
    

    # --- administrative fields ---
     "gestor_area_propostas",

    # --- duplicated / rarely useful flags ---
    "hubleads__nao_comunica_com_cc_",
    "converted_contact", "converted_deal", "converted_from_lead_deals",
    "ccupdated_contactos", "myhyundai_contactos",
    "f2_deals", "modelovendaupdate_deals"
]

src_tbl = "sc_gold.Features_Table"

# compute keep list dynamically from live schema
all_cols = spark.table(src_tbl).columns
keep_cols = [c for c in all_cols if c not in cols_to_delete]
keep_sql = ", ".join([f"`{c}`" for c in keep_cols])  # safe quoting

# 2) Snapshot and recreate table with only keep_cols
spark.sql(f"CREATE OR REPLACE TEMP VIEW _src AS SELECT * FROM {src_tbl}")

spark.sql(f"""
CREATE OR REPLACE TABLE {src_tbl}
USING DELTA AS
SELECT {keep_sql} FROM _src
""")

spark.sql("DROP VIEW _src")


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  *,
  CASE
    WHEN month(created_time) IN (12, 1, 2) THEN 'Winter'
    WHEN month(created_time) IN (3, 4, 5) THEN 'Spring'
    WHEN month(created_time) IN (6, 7, 8) THEN 'Summer'
    ELSE 'Autumn'
  END AS Season
FROM sc_gold.Features_Table;

--I created a Season for ML Models ( thinking in the prediction already)


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  *,
  datediff(created_time_deals, created_time) AS lead_to_deal_days
FROM sc_gold.Features_Table;

-- this is deals date- leads date to give the number of days that take , for my surprise there are negatives but can be typos or retroactive data entry → A deal was created manually before the lead was officially entered in the CRM.


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  *,
  CASE
    WHEN datediff(created_time_deals, created_time) <= 7 THEN 'Fast'
    WHEN datediff(created_time_deals, created_time) <= 30 THEN 'Medium'
    ELSE 'Slow'
  END AS deal_speed_category
FROM sc_gold.Features_Table;



In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  *,
  CASE
    WHEN created_time_deals < created_time THEN 1
    ELSE 0
  END AS deal_before_lead
FROM sc_gold.Features_Table;


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  *,
  CASE
    WHEN dayofweek(created_time) IN (1,7) THEN 1
    ELSE 0
  END AS is_weekend_lead
FROM sc_gold.Features_Table;


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  *,
  CASE
    WHEN day(created_time_deals) >= 25 THEN 1
    ELSE 0
  END AS is_month_end_deal
FROM sc_gold.Features_Table;


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table
SELECT 
    *,
    CASE 
        WHEN qtd_viaturas_deals = 0 OR qtd_viaturas_deals IS NULL 
        THEN 0
        ELSE num_de_propostas_deals / qtd_viaturas_deals
    END AS proposals_per_vehicle
FROM sc_gold.Features_Table;


In [0]:
%sql
CREATE OR REPLACE TABLE sc_gold.Features_Table AS
SELECT
  *,
  year(created_time)  AS lead_year,
  month(created_time) AS lead_month
FROM sc_gold.Features_Table;


In [0]:
%sql
SELECT * FROM sc_gold.Features_Table

In [0]:
%sql
SELECT
  is_converted,
  COUNT(*) AS total_leads
FROM sc_gold.leads_pbs
GROUP BY is_converted
ORDER BY is_converted;



In [0]:
%sql
SELECT
  is_converted,
  COUNT(*) AS total_leads
FROM sc_silver.leads_pbs
GROUP BY is_converted
ORDER BY is_converted;